In [ ]:
#Importing Libraries
import numpy as np
import tensorflow as tf
import pickle #Pickling means it converts the Python objects into byte streams.
 
from tensorflow.keras import layers , activations , models , preprocessing

In [ ]:
a = '/content/new.zip'
!unzip new.zip

Archive:  new.zip
   creating: new/new/
   creating: new/new/data/
  inflating: new/new/data/Acne.yaml  
  inflating: new/new/data/AIDS.yaml  
  inflating: new/new/data/allergy.yaml  
  inflating: new/new/data/Arthritis.yaml  
  inflating: new/new/data/botprofile.yaml  
  inflating: new/new/data/Bronchial Asthma.yaml  
  inflating: new/new/data/cancer.yaml  
  inflating: new/new/data/Cervical spondylosis.yaml  
  inflating: new/new/data/chest_pain.yaml  
  inflating: new/new/data/Chicken pox.yaml  
  inflating: new/new/data/Common Cold.yaml  
  inflating: new/new/data/Dengue.yaml  
  inflating: new/new/data/diabetes.yaml  
  inflating: new/new/data/eye_problem.yaml  
  inflating: new/new/data/fever.yaml  
  inflating: new/new/data/fungal.yaml  
  inflating: new/new/data/gastric problem.yaml  
  inflating: new/new/data/glaucoma.yaml  
  inflating: new/new/data/greetings.yml  
  inflating: new/new/data/heartattack.yaml  
  inflating: new/new/data/Hepatitis B.yaml  
  inflating: new/new/d

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml
import os

In [ ]:
dir_path = 'new/new/data'
files_list = os.listdir(dir_path + os.sep)
 
questions = list()
answers = list()
 
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    
    for con in conversations:
       if len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])
print(questions)
 
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
 
answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
print(answers)
 
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers ) #Updates internal vocabulary based on a list of texts.
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

['What are the symptoms of thyroid', 'What are the precautions taken for fatigue', 'What are the medicines for thyroid', 'What are the symptoms of Hypertension', 'What are the precautions taken for Hypertension', 'What are the medicines for Hypertension', 'What are the symptoms of Tuberculosis problem', 'What are the precautions taken for Tuberculosis', 'What are the medicines for Tuberculosis', 'who are you', 'What are your interests', 'What are your favorite subjects', 'What are your interests', 'What is your number', 'What is your number', 'What is your favorite number', 'What can you eat', "Why can't you eat food", 'What is your location', 'What is your location', 'Where are you from', 'Where are you', 'Do you have any brothers', 'Do you have any brothers', 'Who is your father', 'Who is your mother', 'Who is your boss', 'What is your age', 'What is your age', 'What are the symptoms of Bronchial Asthma', 'What are the precautions taken for Bronchial Asthma', 'What are the medicines 

In [ ]:
from gensim.models import Word2Vec
import re

vocab=[]
for word in tokenizer.word_index:
    vocab.append(word)
x = max(vocab, key=len)
print(vocab.index(x))


def tokenize(sentences):
    tokens_list=[]
    vocabulary=[]
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '^a-zA-Z',' ',sentence)
        tokens = sentence.split()
        vocabulary+=tokens
        tokens_list.append(tokens)
       
    return tokens_list,vocabulary
    

p=tokenize(questions+answers)
model = Word2Vec(p[0])

tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array( padded_questions )
print(encoder_input_data.shape, maxlen_questions)

tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i]=tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
one_hot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)


910
(312, 54) 54
(312, 55) 55
(312, 55, 1134)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 910 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 910 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 910 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 910 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 910)    1031940     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 910)    1031940     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=50 ) 
model.save( 'model.h5' )

Epoch 1/50
7/7 [==============================] - 52s 6s/step - loss: 1.3414 - accuracy: 0.1048
Epoch 2/50
7/7 [==============================] - 45s 6s/step - loss: 1.1356 - accuracy: 0.0955
Epoch 3/50
7/7 [==============================] - 45s 6s/step - loss: 1.0559 - accuracy: 0.1614
Epoch 4/50
7/7 [==============================] - 45s 6s/step - loss: 1.0151 - accuracy: 0.1959
Epoch 5/50
7/7 [==============================] - 45s 6s/step - loss: 0.9894 - accuracy: 0.1989
Epoch 6/50
7/7 [==============================] - 45s 6s/step - loss: 0.9979 - accuracy: 0.2013
Epoch 7/50
7/7 [==============================] - 45s 6s/step - loss: 0.9594 - accuracy: 0.1998
Epoch 8/50
7/7 [==============================] - 45s 6s/step - loss: 0.9360 - accuracy: 0.2069
Epoch 9/50
7/7 [==============================] - 45s 6s/step - loss: 0.9051 - accuracy: 0.2195
Epoch 10/50
7/7 [==============================] - 45s 6s/step - loss: 0.8774 - accuracy: 0.2249
Epoch 11/50
7/7 [======================

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 914 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 914 ,))
    
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

def get_responses(msg) :
    states_values = enc_model.predict( str_to_tokens(msg) ) 
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
            else :
                decoded_translation += ' {}'.format( 'i do not understand' )
              
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

            
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    return decoded_translation


import tkinter
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Welcome", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
        
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
             
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()

TclError: ignored